In [1]:
# go wide screen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Batch cube stacking

In [2]:
%matplotlib inline
import sys
import os
import os.path
import subprocess
import numpy as np

import tables as tb
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from astropy.visualization import ZScaleInterval

In [3]:
# this is so that in "compute" all imports are found
sys.path.append("/home/idies/workspace/Storage/mxhf/persistent/mygama09")

In [4]:
from hetdex_api.shot import *

# Cube stacking code

In [5]:
COMMANDLINE = False

In [6]:
# coding: utf-8
from __future__ import print_function

import pylab
from astropy.io import fits
import sys

from scipy import optimize
import spectrum
import numpy
import optparse
import PolygonIntersect
from matplotlib import pyplot as plt
import spectrum
from astropy.io import ascii
from astropy.table import Table
import os
import glob
import numpy as np

from scipy.interpolate import UnivariateSpline

import sys
from astropy.table import Column, vstack
from astropy.io import ascii

import pickle

from astropy.table import Table
import tables as tb


def circle(x,y,size):
        tt = np.arange(0.,2.*np.pi, np.pi/18.)
        xx = size/2. * np.cos(tt) + x
        yy = size/2. * np.sin(tt) + y
        return xx,yy

def pixel(x,y,size):
    xx,yy=[],[]
    xx.append(x-size/2.)
    yy.append(y-size/2.)

    xx.append(x+size/2.)
    yy.append(y-size/2.)

    xx.append(x+size/2.)
    yy.append(y+size/2.)

    xx.append(x-size/2.)
    yy.append(y+size/2.)

    return xx,yy

def create_3D_header(xc, yc, rad, decd, pixelsize,start,step):
    h = fits.Header()
    #h.update("WCSDIM  ", 3, "");
    #h.update("WAT0_001", "system=image", "");
    h["CTYPE3"] = "Wave"
    h["CRPIX3"] = 1.
    h["CRVAL3"] = start
    h["CDELT3"] = step
    #h["WAT1_001"] = wtype=tan
    h["CTYPE1"] = "RA---TAN"
    h["CRPIX1"] = xc
    h["CRVAL1"] = rad
    h["CDELT1"] = - pixelsize/3600.
    h["CUNIT1"] = "deg"
    #h["WAT2_001"] = wtype=tan
    h["CTYPE2"] = "DEC--TAN"
    h["CRPIX2"] = yc
    h["CRVAL2"] = decd
    h["CDELT2"] = pixelsize/3600.
    h["CUNIT2"] = "deg"
    return h

def create_2D_header(xc, yc, rad, decd, pixelsize):
    h = fits.Header()
    #h.update("WAT1_001", "wtype=tan axtype=ra", "");
    h["CTYPE1"] = "RA--TAN"
    h["CRPIX1"] = xc
    h["CRVAL1"] = "rad"
    h["CDELT1"] = - pixelsize/3600.
    h["CUNIT1"] = "deg"
    #h["WAT2_001"] = wtype=tan
    h["CTYPE2"] = "DEC--TAN"
    h["CRPIX2"] = yc
    h["CRVAL2"] = decd
    h["CDELT2"] = pixelsize/3600.
    h["CUNIT2"] = "deg"
    return h

def create_I_header():
    h = fits.Header()
    #h.update("WAT1_001", "wtype=tan axtype=ra", "");
    h["CTYPE1"] = "pixel"
    h["CRPIX1"] = 1
    h["CRVAL1"] = 1.
    h["CDELT1"] = 1.
    h["CUNIT1"] = "px"
    #h["WAT2_001"] = wtype=tan
    h["CTYPE2"] = "fiber"
    h["CRPIX2"] = 1
    h["CRVAL2"] = 1.
    h["CDELT2"] = 1.
    h["CUNIT2"] = "fib"
    return h

def tan_dir_sci(RA0, DEC0, PA0, RA,DEC, quiet=True):
    """
    Calculates pixel positions in the IFU for a given set of RA and DEC coordinates.

    Input

    IFU_RA0, IFU_DEC0 = IFU zero coordinates (in deg)
    RA, DEC = 1D arrays of RA and DEC coordinates
    astrom_terms = parameters for the astrometric solution
    quiet = Boolean, disable output (standard: False)

    Returns:
    pixx, pixy = two 1D arrays containing x and y coordinates (in deg!) for the given RAs and DECs
    """
    if not quiet: print("IFU_RA0, IFU_DEC0 = {},{}".format( IFU_RA0, IFU_DEC0) )

    rRA0  = RA0*np.pi/180.
    rDEC0  = DEC0*np.pi/180.

    rPA0 = PA0*np.pi/180.

    if not quiet: print("[tan_dir_sci] IFU_RA0, IFU_DEC0 = {},{}".format( IFU_RA0, IFU_DEC0 ) )

    rRA = RA*np.pi/180.

    rDEC = DEC*np.pi/180.

    if not quiet: print("[tan_dir_gui] rRA, rDEC: {}, {}".format( rRA, rDEC ) )

    # eq 9 in Greisen AIPSMEMO27
    L = np.cos(rDEC)*np.sin(rRA - rRA0)/  (np.sin(rDEC)*np.sin(rDEC0) + np.cos(rDEC)*np.cos(rDEC0)*np.cos(rRA - rRA0))

    M = (np.sin(rDEC)*np.cos(rDEC0) - np.cos(rDEC)*np.sin(rDEC0)*np.cos(rRA - rRA0))/(np.sin(rDEC)*        
                                                np.sin(rDEC0) + np.cos(rDEC)*np.cos(rDEC0)*np.cos(rRA - rRA0))

    # eq 5 in Greisen AIPSMEMO27
    pixx = L*np.cos(rPA0) + M*np.sin(rPA0)
    pixy = M*np.cos(rPA0) - L*np.sin(rPA0)

    return - pixx/np.pi*180.*3600., pixy/np.pi*180.*3600.


def findZeroPixRaDec(x,y, RA0, DEC0):
    """
    Find RA and DEC coordinates that correspond to the given
    pixel coordinates.
    We do this here the cheap way. We use a nonlinear fit 
    rather than working out the inverse transformation.
    """
    def peval(p,RA0, DEC0):
        RA,DEC = p
        return tan_dir_sci(RA0, DEC0, 0., RA, DEC, quiet=True)

    def resid(p,x,y,RA0, DEC0):
        xt,yt = peval(p,RA0, DEC0)
        return (xt-x), (yt-y)

    p0 = [RA0,DEC0]
    bestfit = optimize.leastsq(resid, p0, args=(x,y,RA0,DEC0) )
    return bestfit[0]

def hms2deg(hms):
    tt = hms.split(":")
    h,m,s = float(tt[0]), float(tt[1]), float(tt[2])
    return h*15. + m/4. + s/240.

def dms2deg(dms):
    tt = dms.split(":")
    d,m,s = float(tt[0]), float(tt[1]), float(tt[2])
    return d + m/60. + s/3600.


In [7]:

if COMMANDLINE:
    import argparse

    parser = argparse.ArgumentParser(description='Build a hetdex cube.')
    #parser.add_argument('--basepath', default="/work/03946/hetdex/maverick/red1/reductions")
    parser.add_argument('--basepath', default="../reductions")
    parser.add_argument('--pa', type=float, default=0.,
                                help='Position angle for cube.')

    parser.add_argument('--write_single_cubes', action="store_true",
                                help='Write individual per-shot cubes before median stacking.')

    parser.add_argument('--norm_smoothing', type=float, default=0.005,
                                help='Smoothing for cross IFU and cross exposure fiber to fiber normalisation (default 0.05)')

    parser.add_argument('--ifuslot', type=str, default = "022", nargs='+', metavar='SLOTS',
            help='IFUslot to create cube for, can pass multiple. ')
    
    parser.add_argument('--shotlist', type=str,
                                help='List of actual shots to use.')

    args = parser.parse_args()

    pa=args.pa
    fiberpos = args.dither_use
    ifuslot = args.ifuslot
    basepath = args.basepath
    shotlist = args.shotlist
    write_single_cubes = args.write_single_cubes
    
    
prefix = ""
extensions = ["sky_subtracted", "sky_spectrum", "fiber_to_fiber"]

# read dithall.use
filebase = {}

exposure_times = []

sky_spectra = [] # holds for each fiber spectrum
                 # the corresponing amplifier wide sky (median accorss all fibers after correcting for fiber_to_fiber)
    
# generic remedy virus wavelength grid
wlgrid = np.arange( 1036 ) * 2. + 3470.
wlstart, wlstop = wlgrid[0], wlgrid[-1]
        


fid = -1

## Loading all spectra for all IFUs in all shots that we plan to combine into a cube here

In [8]:
def fix_ifuslot_swap(hdf5file, t):
    h,tl = os.path.split(hdf5file)
    tt = tl.split("_")
    
    date = int(tt[0])
    
    if (date > 20191201) * (date < 20200624):
        ii = t["ifuslot"] == 24
        jj = t["ifuslot"] == 15
        t["ifuslot"][ii] == 15
        t["ifuslot"][jj] == 24
        
    if (date > 20200114) * (date < 20200128):
        ii = t["ifuslot"] == 78
        t["ifuslot"][ii] == 79

    return t


def load_data(basepath, shotlist, ifuslots):
    names = ["count", "amplifier", "fiberid", "ra", "dec", "shot", "night", "shotid", "exp"]
    dtype = [int, 'U2', int, float, float, 'U12', 'U8', 'U3', float]
    fibers = Table(names=names, dtype=dtype)

    spectra = {}
    allspec = []

    count = 0
    for shot in shotlist:
        print("Loading shot {}".format(shot))
        hdf5_filename = '{}.h5'.format(shot).replace("-","_")
        if not os.path.exists(os.path.join(basepath, hdf5_filename)):
            print("ERROR: File {} does not exist".format(os.path.join(basepath,hdf5_filename)))
            continue
        hdf5file = tb.open_file(os.path.join(basepath,hdf5_filename) )

        h5fibers = hdf5file.root.Fibers
        h5info   = hdf5file.root.Info

        t = Table(h5info.read())
        fix_ifuslot_swap(hdf5_filename, t)

        ifiber = np.arange(len(h5fibers))

        uifus = np.unique( t["ifuslot"] ).tolist()

        for ifu in ifuslots:
            if not int(ifu) in uifus:
                print("WARNING: Shot {} contains no data for IFU slot {}".format(shot, ifu))
                continue
            print("  IFU slot {}".format(ifu))
            # find all spectra that belong to current IFU
            ii = t["ifuslot"] == int(ifu)
            s = []
            for i in ifiber[ii]:
                x,y = t["ra"][i], t["dec"][i]
                allspec.append( h5fibers[i]["spectrum"]  )


                amplifier = ""
                night = shot[:8]
                shotid = shot[9:]
                fibers.add_row([count, amplifier, i, x,y, shot, night, shotid, 0 ])

        print("len(allspec): ", len(allspec))
        print("len(fibers): ", len(fibers))

    allspec = np.array(allspec)
    allspec[np.isnan(allspec)] = 0.
    print("Done.")
    
    return fibers, allspec

#fibers, allspec = load_data(shotlist, ifuslots)

## calculate pixel/fiber intersections

In [9]:
import multiprocessing
from joblib import Parallel, delayed

num_cores = multiprocessing.cpu_count()

print("Have {} cores.".format(num_cores))

Have 12 cores.


In [10]:
def calculate_pixel_fiber_intersec(shotlist, fibers, pixelsize, FIBERD, MULTIPROCESSING = True):
    # USING SPARSE MATRICES!!!
    from scipy.sparse import lil_matrix as sparse_matrix
    from scipy.sparse import csr_matrix

    #csr_matrix

    #from numpy import array as sparse_matrix
    import time

    print( "Calculating fiber/pixel weight maps..." )

    nI = {}

    manager = multiprocessing.Manager()
    nI = manager.dict()
    jobs = []


    def worker(shot, shotfibers, pixels, shotfxx, shotfyy, pixelsize, FIBERD, nI):
        I = sparse_matrix( np.zeros([len(pixels),len(shotfibers)], dtype=np.float) ) # I is the pixel fiber intersection matrix
        #I = np.array( np.zeros([len(pixels),len(shotfibers)], dtype=np.float) ) # I is the pixel fiber intersection matrix

        fib_range = numpy.arange( len(shotfibers) )
        pix_range = range( len(pixels) )

        NNN = 0

        start_time = time.time()
        npix = len(pix_range)

        for ip in pix_range:
            if ip % 1000 == 0:



                s = "  {} pixel: {:6d} of {:6d} ".format(shot, ip, npix)
                sys.stdout.write( '\r'* len(s))
                sys.stdout.write(s)
                sys.stdout.flush()

            p = pixels[ip]

            px,py = p[1],p[2]
            #calculate distances of all fibers of this shot to the current pixel
            dd_sq = (shotfxx-px)**2. + (shotfyy-py)**2. 

            # Find which fibers of this shot could possibly intersect with the current pixel
            # in the intersection filter, only shotfibers which overlap the pixel are considered.
            # We only look at shotfibers wich are not further than 
            # sqrt(2) * pixelsize/2 + fiberd/2 
            ii = ( dd_sq < (pixelsize/2. *  1.414 + FIBERD/2.)**2.)
            # create a polygon describing the current pixel
            ppxx,ppyy = pixel(px,py,pixelsize)
            pixel_poly = list( zip(ppxx,ppyy) )

            if any(ii):
                for ifib in fib_range[ii]:
                    fx = shotfxx[ifib]
                    fy = shotfyy[ifib]

                    NNN += 1
                    fpxx,fpyy = circle(fx,fy,FIBERD)
                    fiber_poly = list( zip(fpxx,fpyy) )
                    fiber_array = PolygonIntersect.toPointsArray(fiber_poly)
                    pixel_array = PolygonIntersect.toPointsArray(pixel_poly)
                    # calculate intersection area
                    iA = PolygonIntersect.intersectionArea(fiber_array, pixel_array)
                    # Now, the flux of a given fiber (at a given wavelength)
                    # will be assigned to a pixel weighted by the fraction of the 
                    # total fiber area that is overlapping with the pixel.
                    I[ip,ifib]  = iA#/fiberA


        #nI[shot] =  I.T/fiberA 
        nI[shot] =  csr_matrix( I.T/fiberA ) 

        print("Done.")

        elapsed_time = time.time() - start_time
        print("elapsed_time: {:.2f}".format(elapsed_time) )



    for shot in shotlist:
        print("Launching shot {}".format(shot))

        jj = fibers["shot"] == shot
        shotfibers = fibers[jj]
        shotfxx = fxx[jj]
        shotfyy = fyy[jj]

        if MULTIPROCESSING:
            p = multiprocessing.Process(target=worker, args=(shot, shotfibers, pixels, shotfxx, shotfyy, pixelsize, FIBERD, nI))
            jobs.append(p)
            p.start()
        else:
            worker(shot, shotfibers, pixels, shotfxx, shotfyy, pixelsize, FIBERD, nI)

    if MULTIPROCESSING:
        for proc in jobs:
            proc.join()

    print("All done.")

    return nI

# nI = calculate_pixel_fiber_intersec(shotlist, fibers, pixelsize, FIBERD)

## Create Cube

In [11]:
def build_cubes(shotlist, fibers, allspec, nI, X,Y,nx, ny, pixelsize, fiberA, wlgrid, wlstart, wlstop):
    print("Creating cube(s)...")
    cube = {}
    W = {}

    for shot in shotlist:
        print("shot {}".format(shot))
        shotspec = allspec[fibers["shot"] == shot]
        ##  shotnormalisations = normalisations[fibers["shot"] == shot]
        # with the help of the intersection matix
        # the pixel values of each wavelength slice are simply the dot product of that matrix (transposed) in the vector
        # of all the fiber values at a give wavelength.
        cube[shot] = np.zeros( [shotspec.shape[1],ny,nx] )

        #W[shot] = np.sum(nI[shot]*fiberA,axis=0)/(pixelsize**2.)
        # this bacame necessary when convarting to sparce matrices
        W[shot] = np.array( np.sum(nI[shot]*fiberA,axis=0)/(pixelsize**2.) )[0]

        # calculate median flux level of all fibers
        kk = (wlgrid > wlstart) * (wlgrid < wlstop) 
        mm = np.median(shotspec[:,kk],axis=1)

        nshotspec = (shotspec.transpose()/mm).transpose()

        IT = nI[shot].transpose()

        def stats(a):
            print("min,max = ", a.min, a.max() )
            print("mean = ", mean(a) )
            print("std = ", std(a) )

        if True:
            for iwl in range(shotspec.shape[1]): # for each wavelength

                if iwl % 100 == 0:
                    s = "WL: %6d %14.6f" % ( iwl,wlgrid[iwl] )
                    sys.stdout.write( '\r'* len(s))
                    sys.stdout.write(s)
                    sys.stdout.flush()

                if True:
                    # usual fast way by matrix multiplication
                    imf2 = IT.dot(shotspec[:,iwl])
                    ii = W[shot] > 0. # prevent zero div error
                    #1/0
                    imf2[ii] = imf2[ii]/W[shot][ii]

                    im2 = imf2.reshape(X.shape)

                    PLOT = False 
                    if PLOT:
                        # PLOT
                        X=0.
                        Y=0.
                        vmin=-20.
                        vmax=20.
                        cmap =  plt.cm.jet 
                        s = plt.subplot(111)
                        dd = np.sqrt((fxx-X)**2. + (fyy-Y)**2.)
                        jj = dd < 50.
                        for ifib,fx,fy in zip(fibers["count"][jj], fxx[jj],fyy[jj]):
                            fpxx,fpyy = circle(fx,fy,FIBERD)
                            #s.plot([fx],[fy],'k.')
                            val = np.nanmedian( shotspec[int(ifib),100:-100] )
                            c = (val-vmin)/(vmax-vmin)

                            s.fill( fpxx,fpyy, facecolor=cmap(c),edgecolor='None',linewidth=1.)
                            #s.text(fx,fy,"{:d}".format(int(ifib) ))
                        plt.show()

                cube[shot][iwl] = im2

            print("")

    # In[156]:


    wstart = wlgrid[0]
    wstep  = wlgrid[1]-wlgrid[0]
    return cube, W, wstart, wstep


#cube, W, wstart, wstep = \
#    build_cubes(shotlist, fibers, allspec, nI, X,Y,nx, ny, pixelsize, fiberA, wlgrid, wlstart, wlstop)

## save output

In [12]:
def stack_cube(cube, shotlist):
    print("Stacking {} cubes ...".format(len(cube)))

    import numpy as np
    cc = np.array( [cube[shot] for shot in shotlist] )


    cc[cc == 0.] = np.nan

    mc = np.zeros_like(cc[0])

    ymin  = 0

    N = 20
    while ymin < mc.shape[1]:
        xmin  = 0
        while xmin < mc.shape[1]:    
            mc[:,ymin:ymin+N,xmin:xmin+N] = np.nanmedian( cc[:,:,ymin:ymin+N,xmin:xmin+N], axis=0 )
            xmin += N
        ymin += N

    cc[cc == 0.] = np.nan

    cc[np.isnan(cc)] = 0.
    return cc, mc
    
def save_cube(mc, xx, yy, pixelsize, RA0, DEC0, wstart, wstep, outfilename):
    print("Saving cube to {}.".format(outfilename))
    #xc,yc are the cube pixel indices that
    #correspond to RA0 and DEC0 and x = 0" and y = 0"
    xc = -xx[0]/pixelsize + 1 
    yc = -yy[0]/pixelsize + 1 

    h2d = create_2D_header(xc, yc, RA0, DEC0, pixelsize)
    h = create_3D_header(xc, yc, RA0, DEC0, pixelsize,wstart,wstep)

    hdu = fits.PrimaryHDU(mc,h)

    hdu.writeto(outfilename,overwrite=True)
    print("Wrote {}.".format(outfilename))
    
#outfilename = "outcube_{}_{}.fits.gz".format("median","-".join(ifuslots))
#save_cube(cube, xx, yy, pixelsize, RA0, DEC0, wstart, wstep, outfilename)

# show some slice

In [13]:
if False:
    c = spectrum.readSpectrum(outfilename)

    from matplotlib import pyplot as plt

    plt.imshow(c.data[740])

In [14]:
## determine a pixel grid

def find_pixel_grid(fibers, RA0=None, DEC0=None, nx = None, ny = None, PLOT = False, RA0_DEC0_ALIGN=True):
    
    if RA0 == None or DEC0 == None:
        RA0, DEC0 = numpy.mean(fibers["ra"]), numpy.mean(fibers["dec"])
    print("Set tangent point for projection to RA0 = %.6f and DEC0  = %.6f." % (RA0, DEC0)) 

    fxx,fyy = tan_dir_sci(RA0, DEC0, pa, fibers["ra"], fibers["dec"], quiet=True)



    maxx = max(fxx)+FIBERD/2.
    maxy = max(fyy)+FIBERD/2.
    minx = min(fxx)-FIBERD/2.
    miny = min(fyy)-FIBERD/2.

    print("Extent in RA : {:.1f} \"".format(maxx-minx))
    print("Extent in Dec : {:.1f} \"".format(maxy-miny))
    print("minx,maxx: ", minx, maxx)
    print("miny,maxy: ", miny, maxy)
    print("fxx: ", len(fxx))
    print("fyy: ", len(fyy))

    if nx == None or ny == None:
        # Automatically determine pixel gridsize,
        # in this new version we make sure that the central pixel sits exactly on RA0 and DEC0
        nx = int( round( ( maxx - minx ) / pixelsize ))
        ny = int( round( ( maxy - miny ) / pixelsize ))
        # create list of all pixel center coordinates 

        if RA0_DEC0_ALIGN:
            # shift minx and miny such that the pixel
            # are aligned with RA0 DEC0
            _minx = int(np.round(minx/pixelsize))*pixelsize
            _miny = int(np.round(miny/pixelsize))*pixelsize
            xx=np.arange(nx)*pixelsize + _minx #+ pixelsize/2.
            yy=np.arange(ny)*pixelsize + _miny #+ pixelsize/2.             
        else:
            # old version
            xx=np.arange(nx)*pixelsize + minx + pixelsize/2.
            yy=np.arange(ny)*pixelsize + miny + pixelsize/2. 
    else:
        # use user-defined grid instead
        xx=np.arange(nx)-(nx-1.)/2.
        yy=np.arange(ny)-(ny-1.)/2.
        xx *= pixelsize
        yy *= pixelsize

    X,Y=np.meshgrid(xx,yy)
    pixels = np.zeros( [ len(X.flatten()) ,3]  ) 
    pixels[:,0] = np.arange(len(pixels))
    pixels[:,1] = X.flatten()
    pixels[:,2] = Y.flatten()

    PLOT = False
    if PLOT:
        # plotting
        s = plt.subplot() 
        #for p in pixels:
            #    xx,yy = pixel(p[1],p[2], pixelsize)
        #    s.fill( xx, yy, facecolor='none',linewidth=0.2 )

        for f in zip( fxx,fyy ):
            xx,yy = circle(f[0],f[1], FIBERD)
            s.fill( xx, yy, facecolor='none',linewidth=0.2,edgecolor='blue' )

        s.set_xlabel("x (\")")
        s.set_ylabel("y (\")")
        s.axis('equal')
        
    return pixels,xx,yy,X,Y,fxx,fyy,nx,ny, RA0, DEC0

## Run over multiple IFUs

In [15]:
RA0 = None 
DEC0 = None
FIBERD = 1.5
nx = None
ny = None
pixelsize = .5
fiberA = np.pi*(FIBERD/2.)**2.
write_single_cubes = False    
pa = 0.


RA0= 130.161884
DEC0  = 2.366698

RA0   = 130.05
DEC0  = 2.35 
        

In [16]:
basepath = "/home/idies/workspace/Storage/mxhf/persistent/mygama09/gama_recon/"
outpath  = "/home/idies/workspace/Storage/mxhf/persistent/mygama09/data"
SKIP_EXISTING = True

In [40]:
shotlist = {}
__ = """20191203_0000024
20191221_0000022
20191203_0000025
20191222_0000023
20191221_0000023
20191224_0000024
20191222_0000024
20191229_0000023
20191231_0000024
20200101_0000019
20191231_0000025
20200101_0000020"""
shotlist["gama09E"] = __.split()

# new
shotlist = {}
__ = """20191203-0000024
20191221-0000022
20191203-0000025
20191222-0000023
20210204-0000016
20191221-0000023
20191224-0000024
20191222-0000024
20191229-0000023
20210208-0000015
20191231-0000024
20200101-0000019
20191231-0000025
20200101-0000020
20210113-0000021"""
shotlist["gama09Efin"] = __.split()

__ = """20200118_0000017
20200215_0000016
20200119_0000018
20200217_0000014
20200124_0000016
20200217_0000015
20200119_0000019
20200225_0000016
20200125_0000020
20200315_0000012
20200126_0000020"""
shotlist["gama09F"] = __.split()

# new
__ = """20200118-0000017
20200215-0000016
20200119-0000018
20200217-0000014
20210205-0000016
20200124-0000016
20200217-0000015
20200119-0000019
20200225-0000016
20200125-0000020
20200315-0000012
20200126-0000020
20201217-0000019
20210114-0000016"""
shotlist["gama09Ffin"] = __.split()

__ = """
20200126-0000021
20201113-0000027
20200127-0000018
20201113-0000028
20210207-0000014
20200127-0000019
20201115-0000023
20200129-0000018
20201116-0000030
20200213-0000015
20201217-0000020
20200129-0000019
20201219-0000020
20210116-0000019"""
shotlist["gama09Gfin"] = __.split()

__ = """
20201212-0000026
20201220-0000024
20201214-0000019
20201222-0000022
20210208-0000014
20201214-0000019
20201222-0000022
20201215-0000020
20210107-0000021
20201216-0000023
20210108-0000021
20201216-0000024
20201219-0000022
20210115-0000016"""
shotlist["gama09Hfin"] = __.split()

fields = ["gama09Efin", "gama09Ffin", "gama09Gfin", "gama09Hfin"]
#fields = ["gama09Ffin"]

In [18]:
#old
#uifuslots = [13, 14, 15, 16, 21,   \
#             22, 23, 24, 25, 26,   \
#             27, 28, 31, 32, 33,   \
#             34, 35, 36, 37, 38,   \
#             41, 42, 43, 44, 45,   \
#             46, 47, 48, 52, 53,   \
#             62, 63, 67, 68, 71,   \
#             72, 73, 74, 75, 76,   \
#             77, 79, 81, 82, 83,   \
#             84, 85, 86, 87, 88,   \
#             91, 92, 93, 94, 95,   \
#             96, 97, 98, 103, 104, \
#             105, 106]



In [19]:
uifuslots = {}


for field in fields:
    allslots = []
    for shot in shotlist[field][:]:
        file = '{}.h5'.format(shot.replace("-","_"))
        filepath = os.path.join(basepath, file)
        if not os.path.exists(filepath):
            print("Warning {} does not exist.".format(filepath))
            continue
        fileh = tb.open_file( filepath )
        t = Table(fileh.root.Info.read())

        allslots += np.unique( t["ifuslot"] ).tolist() 

        allslots = np.sort( np.unique(allslots) ).tolist() 
        #print("after adding slots from ", shot, " have ", len(allslots), ' slots.')
        
        uifuslots[field] = allslots
    print(allslots)
print(uifuslots)

[13, 14, 15, 16, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 52, 53, 57, 58, 61, 62, 63, 67, 68, 71, 72, 73, 74, 75, 76, 77, 78, 79, 81, 82, 83, 84, 85, 86, 87, 88, 89, 91, 92, 93, 94, 95, 96, 97, 98, 103, 104, 105, 106]
[13, 14, 15, 16, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 52, 53, 57, 58, 61, 62, 63, 67, 68, 69, 71, 72, 73, 74, 75, 76, 77, 78, 79, 81, 82, 83, 84, 85, 86, 87, 88, 89, 91, 92, 93, 94, 95, 96, 97, 98, 103, 104, 105, 106]
[13, 14, 15, 16, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 52, 53, 57, 58, 61, 62, 63, 67, 68, 69, 71, 72, 73, 74, 75, 76, 77, 78, 79, 81, 82, 83, 84, 85, 86, 87, 88, 89, 91, 92, 93, 94, 95, 96, 97, 98, 103, 104, 105, 106]
[13, 14, 15, 16, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 

In [37]:
uifuslots['gama09Efin'] = [23]

uifuslots['gama09Ffin'] = [23]

uifuslots['gama09Gfin'] = [23]

uifuslots['gama09Hfin'] = [23]

SKIP_EXISTING = False

In [41]:
# compine multiple fields
ifuslots = [23, 24, 33, 34]
ifuslots = [23, 24]

sl =[]
for field in fields:
    sl += shotlist[field] 
    
    
print("###########################")
print("Processing field {} ifus {}".format(field, ifuslots))
print("###########################")
#continue

#outfilename = os.path.join( outpath, "{}_{:03d}.fits.gz".format(field,ifu) )
outfilename = os.path.join( outpath, "test2.fits.gz" )

if os.path.exists(outfilename) and SKIP_EXISTING:
    print("{} exists, skipping ...".format(outfilename))

# load data
fibers, allspec = load_data(basepath, sl, ifuslots)

# compute pith and location of pixel grid

#pixels,xx,yy,X,Y,fxx,fyy,nx,ny, RA0, DEC0 = find_pixel_grid(fibers)


pixels,xx,yy,X,Y,fxx,fyy,nx,ny, RA0, DEC0 = find_pixel_grid(fibers, RA0=RA0, DEC0=DEC0)

# compute fiber to pixel flux mappings
nI = calculate_pixel_fiber_intersec(sl, fibers, pixelsize, FIBERD)

# build cubes
cube, W, wstart, wstep = \
    build_cubes(sl, fibers, allspec, nI, X,Y,nx, ny, pixelsize, fiberA, wlgrid, wlstart, wlstop)

# Free some memory
nI = None 
W = None

# stack & save
cc, mc = stack_cube(cube, sl)

# stack & save
save_cube(mc, xx, yy, pixelsize, RA0, DEC0, wstart, wstep, outfilename)

1/0

###########################
Processing field gama09Hfin ifus [23, 24, 33, 34]
###########################
Loading shot 20191203-0000024
  IFU slot 23
  IFU slot 24
  IFU slot 33
  IFU slot 34
len(allspec):  5376
len(fibers):  5376
Loading shot 20191221-0000022
  IFU slot 23
  IFU slot 24
  IFU slot 33
  IFU slot 34
len(allspec):  10752
len(fibers):  10752
Loading shot 20191203-0000025
  IFU slot 23
  IFU slot 24
  IFU slot 33
  IFU slot 34
len(allspec):  16128
len(fibers):  16128
Loading shot 20191222-0000023
  IFU slot 23
  IFU slot 24
  IFU slot 33
  IFU slot 34
len(allspec):  21504
len(fibers):  21504
Loading shot 20210204-0000016
  IFU slot 23
  IFU slot 24
  IFU slot 33
  IFU slot 34
len(allspec):  26880
len(fibers):  26880
Loading shot 20191221-0000023
  IFU slot 23
  IFU slot 24
  IFU slot 33
  IFU slot 34
len(allspec):  32256
len(fibers):  32256
Loading shot 20191224-0000024
  IFU slot 23
  IFU slot 24
  IFU slot 33
  IFU slot 34
len(allspec):  37632
len(fibers):  37632
Loading

OSError: [Errno 12] Cannot allocate memory

In [ ]:
for field in fields:

    for ifu in uifuslots[field]:
        print("###########################")
        print("Processing field {} ifu {}".format(field, ifu))
        print("###########################")
        #continue
        
        outfilename = os.path.join( outpath, "{}_{:03d}.fits.gz".format(field,ifu) )
        if os.path.exists(outfilename) and SKIP_EXISTING:
            print("{} exists, skipping ...".format(outfilename))
            continue

        # load data
        fibers, allspec = load_data(basepath, shotlist[field], [ifu])

        # compute pith and location of pixel grid

        #pixels,xx,yy,X,Y,fxx,fyy,nx,ny, RA0, DEC0 = find_pixel_grid(fibers)

        
        pixels,xx,yy,X,Y,fxx,fyy,nx,ny, RA0, DEC0 = find_pixel_grid(fibers, RA0=RA0, DEC0=DEC0)

        # compute fiber to pixel flux mappings
        nI = calculate_pixel_fiber_intersec(shotlist[field], fibers, pixelsize, FIBERD)

        # build cubes
        cube, W, wstart, wstep = \
            build_cubes(shotlist[field], fibers, allspec, nI, X,Y,nx, ny, pixelsize, fiberA, wlgrid, wlstart, wlstop)

        # Free some memory
        nI = None 
        W = None

        # stack & save
        cc, mc = stack_cube(cube, shotlist[field])

        # stack & save
        save_cube(mc, xx, yy, pixelsize, RA0, DEC0, wstart, wstep, outfilename)

In [34]:
def stack_cube(cube, sl):
    # this implemtation is much slower, but much leaner on memory.
    shotlist = sl
    print("Stacking {} cubes ...".format(len(cube)))

    import numpy as np
    final_cube = np.zeros_like( cube[sl[0]] )

    for i in range(cube[sl[0]].shape[0]):
        print("Working on wavelength slice {}.".format(i))
        cc = np.array( [cube[shot][i] for shot in sl] )
        cc[cc == 0.] = np.nan

        mc = np.zeros_like(cc[0])

        ymin  = 0

        N = 20
        while ymin < mc.shape[1]:
            xmin  = 0
            while xmin < mc.shape[1]:    
                mc[ymin:ymin+N,xmin:xmin+N] = np.nanmedian( cc[:,ymin:ymin+N,xmin:xmin+N], axis=0 )
                xmin += N
            ymin += N

        cc[cc == 0.] = np.nan

        cc[np.isnan(cc)] = 0.

        final_cube[i] = mc

    return cc, final_cube

Stacking 55 cubes ...
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271


In [36]:
# stack & save
save_cube(final_cube, xx, yy, pixelsize, RA0, DEC0, wstart, wstep, outfilename)

Saving cube to /home/idies/workspace/Storage/mxhf/persistent/mygama09/data/test.fits.gz.
Wrote /home/idies/workspace/Storage/mxhf/persistent/mygama09/data/test.fits.gz.


In [28]:
cc = np.zeros([len(cube), cube[sl[0]].shape[1], cube[sl[0]].shape[2]])

In [29]:
cc.shape

(55, 282, 288)

In [27]:
cube[sl[0]].shape

(1036, 282, 288)

In [25]:
sl

['20191203-0000024',
 '20191221-0000022',
 '20191203-0000025',
 '20191222-0000023',
 '20210204-0000016',
 '20191221-0000023',
 '20191224-0000024',
 '20191222-0000024',
 '20191229-0000023',
 '20210208-0000015',
 '20191231-0000024',
 '20200101-0000019',
 '20191231-0000025',
 '20200101-0000020',
 '20210113-0000021',
 '20200118-0000017',
 '20200215-0000016',
 '20200119-0000018',
 '20200217-0000014',
 '20210205-0000016',
 '20200124-0000016',
 '20200217-0000015',
 '20200119-0000019',
 '20200225-0000016',
 '20200125-0000020',
 '20200315-0000012',
 '20200126-0000020',
 '20201217-0000019',
 '20210114-0000016',
 '20200126-0000021',
 '20201113-0000027',
 '20200127-0000018',
 '20201113-0000028',
 '20210207-0000014',
 '20200127-0000019',
 '20201115-0000023',
 '20200129-0000018',
 '20201116-0000030',
 '20200213-0000015',
 '20201217-0000020',
 '20200129-0000019',
 '20201219-0000020',
 '20210116-0000019',
 '20201212-0000026',
 '20201220-0000024',
 '20201214-0000019',
 '20201222-0000022',
 '20210208-00

In [36]:
%pdb

Automatic pdb calling has been turned ON


In [ ]:
1/0

# Testing of new pixel grid

In [ ]:
shotlist = {}

__ = """20191203-0000024
20191221-0000022
"""
shotlist["gama09Efin"] = __.split()


__ = """20200118-0000017
20210114-0000016"""
shotlist["gama09Ffin"] = __.split()


__ = """
20200126-0000021
20210116-0000019"""
shotlist["gama09Gfin"] = __.split()

__ = """
20201212-0000026
20210115-0000016"""
shotlist["gama09Hfin"] = __.split()



uifuslots['gama09Efin'] = [23]

uifuslots['gama09Ffin'] = [23]

uifuslots['gama09Gfin'] = [23]

uifuslots['gama09Hfin'] = [23]


fields = ['gama09Efin', 'gama09Ffin', 'gama09Gfin', 'gama09Hfin']


SKIP_EXISTING = False


XX = []
YY = []

for field in fields:

    for ifu in uifuslots[field]:
        print("###########################")
        print("Processing field {} ifu {}".format(field, ifu))
        print("###########################")
        #continue
        
        outfilename = os.path.join( outpath, "{}_{:03d}.fits.gz".format(field,ifu) )
        if os.path.exists(outfilename) and SKIP_EXISTING:
            print("{} exists, skipping ...".format(outfilename))
            continue

        # load data
        fibers, allspec = load_data(basepath, shotlist[field], [ifu])

        # compute pith and location of pixel grid

        RA0= 130.161884
        DEC0  = 2.366698
        pixels,xx,yy,X,Y,fxx,fyy,nx,ny, RA0, DEC0 = find_pixel_grid(fibers, RA0=RA0, DEC0=DEC0)
        
        XX.append(X)
        YY.append(Y)


In [ ]:
%matplotlib notebook
f = plt.figure(figsize=[20,20])
for X,Y in zip(XX,YY):
    plt.plot(X.flatten(),Y.flatten(),'.')

plt.axis("equal")
#plt.xlim([-40,-30])

plt.ylim([-20,20])
